## Imports and Setup

Let's start with the base imports.

In [ ]:
import os
import pathlib

import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from six.moves.urllib.request import urlopen

import tensorflow as tf
import tensorflow_hub as hub

tf.get_logger().setLevel('ERROR')
# Print Tensorflow version
print(tf.__version__)

# Check available GPU devices.
print("The following GPU devices are available: %s" % tf.test.gpu_device_name())

## Utilities

- Helper method to load an image

In [ ]:
def load_image_into_numpy_array(path):
  image = None
  if(path.startswith('http')):
    response = urlopen(path)
    image_data = response.read()
    image_data = BytesIO(image_data)
    image = Image.open(image_data)
  else:
    image_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(image_data))
    
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape((1, im_height, im_width, 3)).astype(np.uint8)

## Visualization tools

To visualize the images with the proper detected boxes, keypoints and segmentation, we will use the TensorFlow Object Detection API. To install it we will clone the repo.

In [ ]:
# Clone the tensorflow models repository
!git clone --depth 1 https://github.com/tensorflow/models

Intalling the Object Detection API

In [ ]:
%%capture
%%bash 

sudo apt install -y protobuf-compiler
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .


Now we can import the dependencies we will need later

In [ ]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as utils_ops

%matplotlib inline

### Load label map data (for plotting).

Label maps correspond index numbers to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine.

We are going, for simplicity, to load from the repository that we loaded the Object Detection API code

In [ ]:
PATH_TO_LABELS = './models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

## Loading the selected model from TensorFlow Hub

Here we just need the model handle that was selected and use the Tensorflow Hub library to load it to memory.


In [ ]:
%%capture

print('loading model...')
model_handle = "https://tfhub.dev/tensorflow/efficientdet/d7/1"
hub_model = hub.load(model_handle)
print('model loaded!')

[link text](https://)## Doing the inference

To do the inference we just need to call our TF Hub loaded model.

Things you can try:
* Print out `result['detection_boxes']` and try to match the box locations to the boxes in the image.  Notice that coordinates are given in normalized form (i.e., in the interval [0, 1]).
* inspect other output keys present in the result. A full documentation can be seen on the models documentation page (pointing your browser to the model handle printed earlier)

In [ ]:
# running inference
image_path = "images/bottle_top.jpg"
image_np = load_image_into_numpy_array(image_path)
results = hub_model(image_np)

# different object detection models have additional results
# all of them are explained in the documentation
result = {key:value.numpy() for key,value in results.items()}

## Visualizing the results

Here is where we will need the TensorFlow Object Detection API to show the squares from the inference step (and the keypoints when available).

the full documentation of this method can be seen [here](https://github.com/tensorflow/models/blob/master/research/object_detection/utils/visualization_utils.py)

Here you can, for example, set `min_score_thresh` to other values (between 0 and 1) to allow more detections in or to filter out more detections.

In [ ]:
label_id_offset = 0
image_np_with_detections = image_np.copy()


viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections[0],
      result['detection_boxes'][0],
      (result['detection_classes'][0] + label_id_offset).astype(int),
      result['detection_scores'][0],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=.25,
      agnostic_mode=False)


plt.figure(figsize=(18,12))
plt.imshow(image_np_with_detections[0])
plt.show()

### Detect objects of all images in images directory

In [ ]:
import math
import cv2

image_names = os.listdir("images")
ncols = 3
nrows = math.ceil(len(image_names) / ncols)
figsize = [20, 20]

print(f"Cols: {ncols}; Rows: {nrows}")

fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)


for i, axi in enumerate(ax.flat):
  try:
    # Convert image to tensor
    full_size_image = cv2.imread(os.path.join("images", image_names[i]))
    full_size_image = cv2.cvtColor(full_size_image, cv2.COLOR_BGR2RGB)
    
    image_np = cv2.resize(full_size_image, (224,224), interpolation=cv2.INTER_CUBIC)
    image_np = tf.expand_dims(image_np, axis=0)
    
    # Detect objects in images
    results = hub_model(image_np)
    result = {key:value.numpy() for key,value in results.items()}

    label_id_offset = 0
    image_np_with_detections = np.array(image_np).copy()

    # Draw bounding box 
    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections[0],
        result['detection_boxes'][0],
        (result['detection_classes'][0] + label_id_offset).astype(int),
        result['detection_scores'][0],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=200,
        min_score_thresh=.25,
        agnostic_mode=False)
    
    axi.imshow(image_np_with_detections[0])
  except:
    print(f"Failed at iteration {i} of {len(ax.flat)}")
    pass
  

plt.tight_layout(True)
plt.show()


## Detect in Real-Time (only locally)

In [ ]:
import cv2


cap = cv2.VideoCapture(0)

In [ ]:
while True:
  ret, frame = cap.read()

  (im_width, im_height) = frame.size
  image_np = np.array(frame.getdata()).reshape((1, im_height, im_width, 3)).astype(np.uint8)


  results = hub_model(image_np)
  result = {key:value.numpy() for key,value in results.items()}


  viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                result['detection_boxes'],
                result['detection_classes']+label_id_offset,
                result['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.5,
                agnostic_mode=False)

  cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))

  if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break
